**Direct forecasting**

Forecast the horizon = N based on N different models.

In [40]:
import random
import lightgbm as lgb
import pandas as pd
from datasetsforecast.m4 import M4, M4Info
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import smape

from mlforecast import MLForecast
from mlforecast.lag_transforms import ExponentiallyWeightedMean, RollingMean
from mlforecast.target_transforms import Differences


In [41]:
group = 'Hourly'
await M4.async_download('data', group=group)

df, *_ = M4.load(directory='data', group=group)
df['ds'] = df['ds'].astype('int')

df = df[df.unique_id == 'H1']
df.head()

,unique_id,ds,y
0,H1,1,605.0
1,H1,2,586.0
2,H1,3,586.0
3,H1,4,559.0
4,H1,5,511.0


In [42]:
df.shape

(748, 3)

In [43]:
# ids = df['unique_id'].unique()
# random.seed(0)
# sample_ids = random.choices(ids, k=4)
# sample_df = df[df['unique_id'].isin(sample_ids)]
info = M4Info[group]
horizon = info.horizon

valid = df.groupby('unique_id').tail(horizon)
train = df.drop(valid.index)

In [44]:
train.shape

(700, 3)

In [45]:
def avg_smape(df):
    """Computes the SMAPE by serie and then averages it across all series."""
    full = df.merge(valid)
    return (
        evaluate(full, metrics=[smape])
        .drop(columns='metric')
        .set_index('unique_id')
        .squeeze()
    )


In [67]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    # lags=[24 * (i+1) for i in range(7)],
    lags=[1, 2, 3], 
    # lag_transforms={
    #     1: [RollingMean(window_size=24)],
    #     24: [RollingMean(window_size=24)],
    #     48: [ExponentiallyWeightedMean(alpha=0.3)],
    # },
    num_threads=1,
    # target_transforms=[Differences([24])],
)


In [79]:
horizon = 24
# the following will train 24 models, one for each horizon
individual_fcst = fcst.fit(train, max_horizon=horizon)
individual_preds = individual_fcst.predict(horizon, before_predict_callback=inspect_input)
# avg_smape_individual = avg_smape(individual_preds).rename('individual')

,lag1,lag2,lag3
0,684.0,739.0,752.0


In [69]:
X, y = individual_fcst.preprocess(train, return_X_y=True, as_numpy=True)

In [70]:
len(X)

697

In [75]:
X[len(X)-1]

array([739., 752., 784.])

In [76]:
y[len(X)-1]

np.float64(684.0)

In [77]:
train.tail(5)

,unique_id,ds,y
695,H1,696,790.0
696,H1,697,784.0
697,H1,698,752.0
698,H1,699,739.0
699,H1,700,684.0


In [78]:
# the following will train a single model and use the recursive strategy

def inspect_input(new_x):
    """Displays the model inputs to inspect them"""
    display(new_x)
    return new_x

recursive_fcst = fcst.fit(train)
recursive_preds = recursive_fcst.predict(horizon, before_predict_callback=inspect_input)
# avg_smape_recursive = avg_smape(recursive_preds).rename('recursive')


,lag1,lag2,lag3
0,684.0,739.0,752.0


,lag1,lag2,lag3
0,612.604893,684.0,739.0


,lag1,lag2,lag3
0,569.595843,612.604893,684.0


,lag1,lag2,lag3
0,531.01462,569.595843,612.604893


,lag1,lag2,lag3
0,507.710359,531.01462,569.595843


...

,lag1,lag2,lag3
0,830.8417,790.954066,739.618822


,lag1,lag2,lag3
0,853.105147,830.8417,790.954066


,lag1,lag2,lag3
0,868.293396,853.105147,830.8417


,lag1,lag2,lag3
0,857.802075,868.293396,853.105147


,lag1,lag2,lag3
0,853.441089,857.802075,868.293396


In [20]:
X, y = fcst.preprocess(train, return_X_y=True, as_numpy=True)
X

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.00000000e-02,  3.33333333e-02,  2.80980483e-03],
       [-1.00000000e-01,  1.00000000e-01,  0.00000000e+00, ...,
         5.41666667e-02,  2.50000000e-02,  3.19668634e-02],
       [-1.00000000e-01,  1.00000000e-01,  0.00000000e+00, ...,
         6.25000000e-02,  1.66666667e-02,  5.23768044e-02],
       ...,
       [ 8.00000000e+00,  5.00000000e+00, -9.00000000e+00, ...,
         6.20833333e+00,  2.37500000e+00,  7.50198016e+00],
       [ 1.70000000e+01, -7.00000000e+00,  2.00000000e+00, ...,
         6.54166667e+00,  3.37500000e+00,  3.15138611e+00],
       [ 1.10000000e+01, -6.00000000e+00, -5.00000000e+00, ...,
         5.70833333e+00,  4.08333333e+00,  4.05970277e-01]])

In [ ]:
len(X), len(y)

In [21]:
X[0]

array([0.        , 0.        , 0.        , 0.3       , 0.1       ,
       0.1       , 0.3       , 0.05      , 0.03333333, 0.0028098 ])

In [22]:
y[0]

np.float64(0.09999999999999964)

In [ ]:
# results
print('Average SMAPE per method and serie')
avg_smape_individual.to_frame().join(avg_smape_recursive).applymap('{:.1%}'.format)